# Setup

In [2]:
!pwd

/home/umni2/a/umnilab/users/verma99/mk/spr_4711/code


In [1]:
from setup import *

# Get POIs

## SafeGraph

In [3]:
SP.start()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 16:06:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/02 16:06:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/02 16:06:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [22]:
def read_sg_pois(rgn_code=None):
    files = sorted(list((SAFEGRAPH / 'us/poi_211104').glob('*.csv.gz')))
    df = SP.read_csv(files, header=True)
    if isinstance(rgn_code, str):
        df = df.filter(f'region == "{rgn_code}"')
    df = df.select('location_name', 'brands', 'top_category', 
                   'sub_category', 'naics_code', 'latitude', 
                   'longitude', 'street_address', 'city')
    return df
    
x = read_sg_pois('IN')
%time x.toPandas().to_csv(U.mkdir(P.data / 'pois/safegraph') / 'indiana.csv', index=False)

CPU times: user 945 ms, sys: 184 ms, total: 1.13 s
Wall time: 10.3 s


In [27]:
df = pd.read_csv(P.data / 'pois/safegraph/indiana.csv').disp()

133,652 rows x 9 cols; Memory: 58.6 MiB


,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city
,<object>,<object>,<object>,<object>,<int64>,<float64>,<float64>,<object>,<object>
0,Julian Insurance Agency,NaN,"Agencies, Brokerages, and Other Insurance Rela...",Insurance Agencies and Brokerages,524210,40.559222,-85.66137,202 W 3rd St,Marion


## OSM

In [23]:
def read_osm_pois():
    df = gpd.read_file(P.data / 'OSM/pois.gpkg')
    return df
    
%time x = read_osm_pois(); x

CPU times: user 4.84 s, sys: 195 ms, total: 5.04 s
Wall time: 5.18 s


,osm_id,code,fclass,name,geometry
0,75423885,2721,attraction,Michigan-Ohio-Indiana tri-state marker,POINT (-84.80598 41.69613)
1,153753182,2204,park,Neavill's Grove,POINT (-85.50993 38.78209)
2,153795896,2401,hotel,Waycross Camp and Conference Center,POINT (-86.34408 39.30556)
3,154353888,2732,ruins,Old Fairfield,POINT (-84.99552 39.51866)
4,180342873,2204,park,Northgate Community Park,POINT (-87.49814 41.53286)
...,...,...,...,...,...
56965,1149302315,2601,bank,Chase,"POLYGON ((-87.33700 41.55136, -87.33700 41.551..."
56966,1149305626,2205,playground,None,"POLYGON ((-86.00010 39.69550, -86.00000 39.695..."
56967,1149307712,2015,graveyard,Henderson Cemetery,"POLYGON ((-85.20349 39.05308, -85.20341 39.053..."
56968,1149448580,2301,restaurant,Clean Eatz,"POLYGON ((-85.20574 41.07578, -85.20550 41.075..."
